In [1]:
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.document.chunking.recursive import RecursiveChunking
from agno.vectordb.qdrant import Qdrant

def ingest_user_documents(user_id: str, folder_path: str):
    collection_name = user_id  # Use user_id as the Qdrant collection name

    knowledge_base = PDFKnowledgeBase(
        path=folder_path,
        vector_db=Qdrant(collection=collection_name, url="http://localhost:6333"),
        reader=PDFReader(chunk=True),
        # chunking_strategy=RecursiveChunking(chunk_size=4000, overlap=300)
    )

    # Load and index the documents into Qdrant (only do once per user or when files change)
    knowledge_base.load(recreate=False)


In [2]:
from agno.knowledge import AgentKnowledge
from agno.embedder.openai import OpenAIEmbedder
from agno.vectordb.qdrant import Qdrant
from agno.agent import Agent
from agno.models.openai import OpenAIChat
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
def create_user_agent(user_id: str):
    collection_name = user_id  # Must match the collection used during ingestion

    vector_db = Qdrant(
        collection=collection_name,
        url="http://localhost:6333"
    )

    knowledge = AgentKnowledge(
        vector_db=vector_db,
        embedder=OpenAIEmbedder()
    )

    agent = Agent(
        model=OpenAIChat(id="gpt-4.1-nano"),
        knowledge=knowledge,
        search_knowledge=True,
        user_id=user_id,
        instructions=[
            "You are a helpful assistant that answers questions using the user's uploaded PDF documents.",
            "Cite document names when helpful.",
            "If the answer is not found, say so politely."
        ]
    )
    return agent


In [6]:
def run_chat():
    user_id = input("Enter your user ID: ").strip()
    folder_path = f"data/{user_id}"  # Make sure this folder exists with PDFs

    if not os.path.exists(folder_path):
        print("❌ No document folder found for this user.")
        return

    # Step 1: Load documents into user-specific Qdrant collection
    ingest_user_documents(user_id, folder_path)

    # Step 2: Create user-specific agent
    agent = create_user_agent(user_id)

    # Step 3: Start chat loop
    print(f"\n🤖 Agent ready for user {user_id}. Ask questions (type 'exit' to quit).")
    while True:
        question = input(f"[{user_id}]> ")
        if question.lower().strip() in {"exit", "quit"}:
            break
        response = agent.run(question)
        print("🧠", response.content)
run_chat()

INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Loading knowledge base

INFO Reading: Introduction

INFO Added 0 documents to knowledge base

INFO Embedder not provided, using OpenAIEmbedder as default.


🤖 Agent ready for user user_123. Ask questions (type 'exit' to quit).
